In [ ]:
from neural_network.nn_manager.TrainManager import TrainManager
from neural_network.store.DBNNSave import DBNNSave
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Input, Conv2D, BatchNormalization
from keras.layers.convolutional import Convolution1D, Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from random import shuffle
import h5py
import numpy as np
import tensorflow as tf
from keras import optimizers

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
np.random.seed(7)

In [ ]:
class LeftRightNN(TrainManager):
    def __init__(self):
        self.path_to_data = './lr-data-size100.hdf5'
        self.prepare_data(self.path_to_data)
        self.batch_size = 32
        self.epochs = 30
        super(LeftRightNN, self).__init__()
        
        self.datagen_train = ImageDataGenerator(
            vertical_flip=True,
            zoom_range=0.1,
            shear_range=0.1,
            rotation_range = 5
        )
        
        self.datagen_val = ImageDataGenerator(
        )
        
    def prepare_data(self, path):
        hdf5_file = h5py.File(path, 'r')
        self.get_handlers(hdf5_file)
        self.load_sizes()
        
    def load_sizes(self):
        self.img_size_1 = self.X_train.shape[1]
        self.img_size_2 = self.X_train.shape[2]
        self.outputs_size = self.Y_train.shape[1]
        self.num_train_samples = self.X_train.shape[0]
        self.num_val_samples = self.X_val.shape[0]
    
    def get_handlers(self, file):
        self.X_train = file['train_data_x']
        self.Y_train = file['train_data_y']
        self.X_val = file['val_data_x']
        self.Y_val = file['val_data_y']
    
    def store_method(self):
        return DBNNSave()

    def train_data_generator(self):
        generator = self._generator(self.X_train, self.Y_train, self.datagen_train)
        return generator

    def test_data_generator(self):
        generator = self._generator(self.X_val, self.Y_val, self.datagen_val)
        return generator
        
    def store_method(self):
        return DBNNSave()

    def create_model(self):
        input_image = Input(shape=(self.img_size_1, self.img_size_2, 1))

        layer = Conv2D(filters=32, kernel_size=(3, 3))(input_image)
        layer = BatchNormalization(axis=1)(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling2D(pool_size=(2, 2))(layer)
        
        layer = Conv2D(filters=32, kernel_size=(3, 3))(layer)
        layer = BatchNormalization(axis=1)(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling2D(pool_size=(2, 2))(layer)
        
        layer = Flatten()(layer)
        
        layer = Dense(32)(layer)
        layer = BatchNormalization(axis=1)(layer)
        layer = Activation('relu')(layer)

        layer = Dense(2)(layer)
        layer = BatchNormalization(axis=1)(layer)
        output_layer = Activation('softmax')(layer)
        model = Model(inputs=input_image, outputs=output_layer)
        model.compile( optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model
    
    
    def _generator(self,X,Y,datagen):
        datagen.fit(X)
        batches = datagen.flow(X,Y, batch_size=self.batch_size, shuffle=False)
        while 1:
             for batch in batches:
                yield batch
        
    def train(self):
        self.train_model(
            self.num_train_samples // self.batch_size,
            self.num_val_samples // self.batch_size,
            epochs=self.epochs
        )




In [ ]:
lr = LeftRightNN()
history = lr.train()

In [ ]:
def test(nn):
    file = h5py.File(nn.path_to_data, 'r')
    X_test = file['test_data_x']
    Y_test = file['test_data_y']
    
    correct = 0
    print(len(X_test))
    for i in range(len(X_test)):
        x_test = X_test[i:i+1]
        y_test = Y_test[i]
        y_score = nn.model.predict(x_test)
        if np.sum(y_score[0]*y_test) > 0.5:
            correct = correct+1
    print(correct/len(X_test))

test(lr)        
